<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC Dashboard
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas%20Dashboard/Naas_Dashboard_Financial_Consolidation.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Naas+Dashboard+-+Financial+Consolidation:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #naasdashboard #plotly #dash #naas #asset #automation #analytics #snippet #datavizualisation

**Authors:** [Alexandre STEVENS](https://www.linkedin.com/in/alexandrestevenspbix/) & [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/) & [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Description:** This notebook provides a comprehensive dashboard for financial consolidation and analysis.

## Input

### Import libraries

In [ ]:
try:
    import dash
except:
    !pip install dash --user
    import dash
from dash import html, dcc, Input, Output, State
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
from naas_drivers import gsheet
from dash_bootstrap_components._components.Container import Container
from plotly.subplots import make_subplots
import random
import naas_data_product

### Setup Variables

In [ ]:
# Inputs
APP_TITLE = "FEC Dashboard"
APP_LOGO = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRszy3bMYjE7-_YpiDDqWrLEGR-wJKEbLyff-VQMJOZoA&s"
output_dir = "/home/ftp/FEC-engine/outputs"
path_ref_entite =  f"{output_dir}/FEC/referentiel_entite"
path_ref_periode =  f"{output_dir}/FEC/referentiel_periode"
path_kpis =  f"{output_dir}/FEC/dataset_kpis"
path_evolution_ca =  f"{output_dir}/FEC/dataset_evolution_ca"
path_tresorerie =  f"{output_dir}/FEC/dataset_tresorerie"
path_charges =  f"{output_dir}/FEC/dataset_charges"
path_bilan =  f"{output_dir}/FEC/dataset_bilan"

# Outputs
DASH_PORT = 8050 # Defining the port of the dashboard

## Model

### Constants

In [ ]:
# Style used for card
CARD_COL_STYLE1 = {"className": "g-2", "xs": 12, "sm": 12, "md": 12, "lg": 2, "xl": 2}

CARD_COL_STYLE2 = {
    #     "className": "gx-5",
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 12,
    "xl": 12,
}

CARD_COL_STYLE3 = {
    #     "className": "gx-5",
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 6,
    "xl": 6,
}

# Color used for card
CARD_COLOR = ["#fbfbfb", "#fbfbfb", "#fbfbfb", "#fbfbfb", "#fbfbfb", "#fbfbfb"]
BACKGROUND_COLOR = "#f8f9fa"

### Data & Charts

#### Dropdown data: Ref Entities

In [ ]:
df_entities = get_last_df(path_ref_entite)
entities = sorted(df_entities["ENTITY"].tolist())
entities

#### Dropdown data: Ref Scenarios

In [ ]:
df_scenarios = get_last_df(path_ref_periode)
scenarios = sorted(df_scenarios["PERIOD"].tolist(), reverse=True)
scenarios

#### KPIS

In [ ]:
df_kpis = get_last_df(path_kpis)
print("Nb row:", len(df_kpis))
df_kpis.head(12)

#### Evolution CA

In [ ]:
df_evolution_ca = get_last_df(path_evolution_ca)
print("Nb row:", len(df_evolution_ca))
df_evolution_ca.head(1)

#### Chart 1 : Evolution CA

In [ ]:
def create_linechart(
    df_init,
    label_col="LABEL",
    value_col="VALUE",
    xaxis_title=None,
    yaxis_title=None,
):
    # Init
    fig = go.Figure()
    df = df_init.copy()
    max_period = df["PERIOD"].max()
    min_period = df["PERIOD"].min()
    df[value_col] = df[value_col] / 1000 #K€
    
    # Data
    df_n = df[df["PERIOD"] == max_period].reset_index(drop=True)
    df_n_1 = df[df["PERIOD"] == min_period].reset_index(drop=True)

    # Add traces
    fig.add_trace(
        go.Scatter(
            name="N",
            x=df_n[label_col],
            y=df_n[value_col],
            marker=dict(size=8, symbol='circle', color="blue"),
            hoverinfo="text",
            hovertext=df_n[label_col].astype(str) + ": " + df_n[value_col].map("{:,.0f} K€".format).str.replace(",", " "),
            line=dict(width=2.5),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="N-1",
            x=df_n_1[label_col],
            y=df_n_1[value_col],
            marker=dict(size=8, symbol='circle', color="orange"),
            hoverinfo="text",
            hovertext=df_n_1[label_col].astype(str) + ": " + df_n_1[value_col].map("{:,.0f} K€".format).str.replace(",", " "),
            line=dict(width=2.5),
        )
    )
    # Calc var
    last_n = df_n.loc[df_n.index[-1], value_col]
    last_n_1 = df_n_1.loc[df_n_1.index[-1], value_col]
    var = last_n - last_n_1
    if var > 0:
        var = f"+{var}"

    # Add figure title
    fig.update_layout(
#         title=f"Ventes : {last_n} € ({var} € vs N-1)",
        title_font=dict(family="Arial", size=18, color="black"),
        legend=
        dict(
            x=0.5, #Modifier la valeur x pour ajuster l'alignement horizontal de la légende (0.0 à gauche, 1.0 à droite)
            y=1, ## Modifier la valeur y pour ajuster l'alignement vertical de la légende (0.0 en bas, 1.0 en haut)
            orientation="h",
            xanchor='center',  # Alignement horizontal de la légende ('left', 'center', 'right')
            yanchor='bottom'  # Alignement vertical de la légende ('top', 'middle', 'bottom')
        ),
        plot_bgcolor="#ffffff",
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=12, color="black"),
        xaxis={"type": "category"},
        margin=dict(
            l=50,  # Marge gauche
            r=50,  # Marge droite
            t=50,  # Marge supérieure
            b=50,  # Marge inférieure
        ),
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title,
        title_font=dict(family="Arial", size=12, color="black"),
    )
    return fig


fig_linechart = create_linechart(
    df_evolution_ca,
    label_col="MONTH_INDEX",
    value_col="VALUE_CUM",
)
fig_linechart

#### Get Position de trésorerie  data

In [ ]:
# Create sample financial data
data_cashin = {
    "ENTITY": ["Société X"] * 12,
    "SCENARIO": ["2022-12"] * 12,
    "LABEL": [
        "01",
        "02",
        "03",
        "04",
        "05",
        "06",
        "07",
        "08",
        "09",
        "10",
        "11",
        "12",
    ],
    "GROUP": ["Encaissements"] * 12,
    "VALUE": [random.randint(0, 1000) for i in range(0, 12)],
}
data_cashout = {
    "ENTITY": ["Société X"] * 12,
    "SCENARIO": ["2022-12"] * 12,
    "LABEL": [
        "01",
        "02",
        "03",
        "04",
        "05",
        "06",
        "07",
        "08",
        "09",
        "10",
        "11",
        "12",
    ],
    "GROUP": ["Décaissement"] * 12,
    "VALUE": [random.randint(0, 500) * -1 for i in range(0, 12)],
}


In [ ]:
df_cashin = pd.DataFrame(data_cashin)
df_cashin

In [ ]:
df_cashout = pd.DataFrame(data_cashout)
df_cashout

In [ ]:
# concat cash in and cash out
df_position = pd.concat([df_cashin, df_cashout])

# rename column GROUP = "Position" and groupby + agg
df_position["GROUP"] = "Trésorerie"
df_position = df_position.groupby(
    ["ENTITY", "SCENARIO", "LABEL", "GROUP"], as_index=False
).agg({"VALUE": "sum"})
df_position

#### Get répartition des charges data

In [ ]:
# Inputs
data_charges = {
    "ENTITY": ["Société X"] * 5,
    "SCENARIO": ["2022-12"] * 5,
    "LABEL": ["Charges 1", "Charges 2", "Charges 3", "Charges 4", "Charges 5"],
    "VALUE": [100, 200, 300, 400, 500],
}

In [ ]:
df_charges = pd.DataFrame(data_charges)
df_charges

In [ ]:
# df_earningBIT = gsheet.connect(SPREADSHEET_URL).get(SHEET_NAME_4)
# print("Nb row:", len(df_earningBIT))
# df_earningBIT.head(6)

#### Get Répartition Bilan data

In [ ]:
# Create sample financial data
data_asset = {
    "ENTITY": ["Société X"] * 5,
    "SCENARIO": ["2022-12"] * 5,
    "LABEL": [
        "Total Actif",
        "Créances clients",
        "Stocks",
        "Immobilisations",
        "Trésorerie",
    ],
    "PARENT": ["", "Total Actif", "Total Actif", "Total Actif", "Total Actif"],
    "CATEGORY": ["Actif"] * 5,
    "VALUE": [35.4, 10.2, 2.5, 17.1, 5.6],
}

data_liability = {
    "ENTITY": ["Société X"] * 5,
    "SCENARIO": ["2022-12"] * 5,
    "LABEL": ["Total Passif", "Capitaux propres", "Dettes", "Emprunts", "Provisions"],
    "PARENT": ["", "Total Passif", "Total Passif", "Total Passif", "Total Passif"],
    "CATEGORY": ["Actif"] * 5,
    "VALUE": [35.4, 16.8, 14.8, 1.2, 2.6],
}

In [ ]:
df_asset = pd.DataFrame(data_asset)
df_asset

In [ ]:
df_liability = pd.DataFrame(data_liability)
df_liability

#### Chart 2 : Position de trésorerie

In [ ]:
def create_barlinechart(
    df_cashin,
    df_cashout,
    df_position,
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            name="Encaissement",
            x=df_cashin["LABEL"],
            y=df_cashin["VALUE"],
            marker=dict(color="#1b7656"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Bar(
            name="Décaissement",
            x=df_cashout["LABEL"],
            y=df_cashout["VALUE"] * -1,
            marker=dict(color="#cd3244"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            name="Position",
            x=df_position["LABEL"],
            y=df_position["VALUE"],
            mode="lines+markers",
            marker=dict(size=8, symbol='circle', color="#46a7f5"),
            line=dict(width=2.5),
        ),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
#         title=f"Trésorerie Disponible : {df_position.loc[df_position.index[-1], 'VALUE']} €",
        title_font=dict(family="Arial", size=18, color="black"),
        legend=
        dict(
            x=0.5, #Modifier la valeur x pour ajuster l'alignement horizontal de la légende (0.0 à gauche, 1.0 à droite)
            y=1, ## Modifier la valeur y pour ajuster l'alignement vertical de la légende (0.0 en bas, 1.0 en haut)
            orientation="h",
            xanchor='center',  # Alignement horizontal de la légende ('left', 'center', 'right')
            yanchor='bottom'  # Alignement vertical de la légende ('top', 'middle', 'bottom')
        ),
        plot_bgcolor="#ffffff",
#         width=1200,
#         height=800,
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=12, color="black"),
        xaxis={"type": "category"},
        margin=dict(
            l=50,  # Marge gauche
            r=50,  # Marge droite
            t=50,  # Marge supérieure
            b=50,  # Marge inférieure
    ),
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title_r,
        title_font=dict(family="Arial", size=12, color="black"),
        secondary_y=False,
    )
    fig.update_yaxes(
        title_text=yaxis_title_l,
        title_font=dict(family="Arial", size=12, color="black"),
        secondary_y=True,
    )
#     fig.update_traces(showlegend=True)
    fig.show()
    return fig


fig_barline_chart = create_barlinechart(
    df_cashin,
    df_cashout,
    df_position,
)

#### Chart 3 : Répartition des charges

In [ ]:
def create_hbarhcart(
    df_charges,
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            name="N",
            x=df_charges["VALUE"],
            y=df_charges["LABEL"],
            orientation="h",
            text=df_charges["VALUE"],
            textposition="auto",
            marker=dict(color="blue")
        )
    )
    
    fig.add_trace(
        go.Bar(
            name="N-1",
            x=df_charges["VALUE"],
            y=df_charges["LABEL"],
            orientation="h",
            text=df_charges["VALUE"],
            textposition="auto",
            marker=dict(color="orange")
        )
    )
    
    total = "{:,.0f} €".format(df_charges['VALUE'].sum()).replace(",", " ")
    
    fig.update_layout(
#     title=f"Total Charges : {total}",
    title_font=dict(family="Arial", size=18, color="black"),
    plot_bgcolor="#ffffff",
    xaxis_title=None,
    xaxis_showticklabels=False,
    yaxis_title=None,
    margin_pad=10,
    bargap=0.1,  # gap between bars of adjacent location coordinates.
    bargroupgap=0.2,# gap between bars of the same location coordinate.
    margin=dict(
        l=50,  # Marge gauche
        r=50,  # Marge droite
        t=50,  # Marge supérieure
        b=50,  # Marge inférieure
    ),
    legend=
        dict(
            x=0.5, #Modifier la valeur x pour ajuster l'alignement horizontal de la légende (0.0 à gauche, 1.0 à droite)
            y=1, ## Modifier la valeur y pour ajuster l'alignement vertical de la légende (0.0 en bas, 1.0 en haut)
            orientation="h",
            xanchor='center',  # Alignement horizontal de la légende ('left', 'center', 'right')
            yanchor='bottom'  # Alignement vertical de la légende ('top', 'middle', 'bottom')
        ),
    
    )
    fig.show()
    return fig

fig_hbarhcart_chart = create_hbarhcart(df_charges)

#### Chart 4 : Répartition Bilan

In [ ]:
def create_treemapchart(
    df_asset,
    df_liability,
    xaxis_title=None,
    yaxis_title_r=None,
    yaxis_title_l=None,
):

    fig = make_subplots(
        cols=2,
        rows=1,
        horizontal_spacing=0.02,
        specs=[[{"type": "treemap", "rowspan": 1}, {"type": "treemap"}]],
    )

    fig.add_trace(
        go.Treemap(
            labels=df_asset["LABEL"],
            parents=df_asset["PARENT"],
            values=df_asset["VALUE"],
            textinfo="label+value+percent parent",
            marker_colorscale="Blues",
            hoverinfo="label+value+percent parent",
            branchvalues="total",
        ),
        row=1,
        col=1,
    )

    fig.add_trace(
        go.Treemap(
            labels=df_liability["LABEL"],
            parents=df_liability["PARENT"],
            values=df_liability["VALUE"],
            textinfo="label+value+percent parent",
            marker_colorscale="oranges",
            hoverinfo="label+value+percent parent",
            branchvalues="total",
        ),
        row=1,
        col=2,
    )

    fig.update_layout(
            margin=dict(
            l=50,  # Marge gauche
            r=50,  # Marge droite
            t=50,  # Marge supérieure
            b=50,  # Marge inférieure
    ),
    
    )
    fig.show()
    return fig

fig_treemap_chart = create_treemapchart(df_asset, df_liability)

### Design App

#### Create Dropdown

In [ ]:
# Entity's dropdown list
dropdown_entity = dcc.Dropdown(
    id="entity",
    options=[{"label": i, "value": i} for i in entities],
    placeholder="Entity",
    value=entities[0],
)

# Scenario's dropdown list
dropdown_scenario = dcc.Dropdown(
    id="scenario",
    options=[{"label": i, "value": i} for i in scenarios],
    placeholder="Scenario",
    value=scenarios[0],
)

#### Create Navbar

In [ ]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=APP_LOGO, height="50")),
                        dbc.Col(dbc.NavbarBrand(APP_TITLE, className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            dbc.Collapse(
                dbc.Nav(
                    [
                        html.Div(
                            [
                                html.Div(className="w-100"),
                                html.Div(className="w-100"),
                                html.Div(dropdown_entity, className="w-100"),
                                html.Div(dropdown_scenario, className="w-100"),
                            ],
                            className="pt-1 pb-1 d-grid gap-2 d-md-flex w-100",
                        )
                    ],
                    className="ms-auto w-100",
                    navbar=True,
                ),
                id="navbar-collapse",
                navbar=True,
                is_open=False,
            ),
        ],
    ),
    color="#181A1C",
    dark=True,
)

#### Create Card

In [ ]:
# Function to create card
def create_card_col(
    title,
    value,
    value_id,
    var,
    var_id
):
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
                    html.P(
                        title,
                        className="card-title",
                        style={
                            "font-size": "15px",
                            "padding-bottom": "5px",
                            "text-align": "center",
                            "font-weight": "bold",
                            "color": "#181A1C",
                        },
                    ),
                    html.P(
                        children=value,
                        className="card-text",
                        style={
                            "font-size": "35px",
                            "text-align": "center",
                            "font-weight": "bold",
                            "color": "#181A1C"
                        },
                        id=value_id,
                    ),
                    
                    html.P(
                        children=var,
                        className="card-text",
                        style={
                            "font-size": "16px",
                            "text-align": "center",
                            "padding-top": "0px",
                            "color": "#181A1C",
                        },
                        id=var_id,
                    ),
                    
                ]
            ),
            color="#fbfbfb",
            inverse=True,
        ),
            
            style={
                "padding-left": "10px", 
                "padding-right": "10px"
            },
        **CARD_COL_STYLE1,
    )
    return card

#### Create Chart Card

In [ ]:
def create_chart(title, subtitle, chart_id, fig, CARD_COL_STYLE):
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
                    html.H5(
                        title,
                        style={
                            "padding-top": "20px",
                            "margin-left": "20px",
                            "font-weight": "bold",
                            "font-size": "17px",
                        },
                    ),
                    
                     html.H5(
                        subtitle,
                        style={
                            "padding-top": "4px",
                            "margin-left": "20.5px",
                            "font-size": "12px",
                        },
                    ),
                    dcc.Graph(
                        id=chart_id,
                        figure=fig,
                        config={
                            "displayModeBar": False,
#                             "staticPlot": True
                        }
                    ),
                ],
            ),
            color=CARD_COLOR,
            inverse=False,
        ),
        style={
#             "background-color": BACKGROUND_COLOR,
            "display": "inline",
            "padding-left": "10px", 
            "padding-right": "10px"
        },
        **CARD_COL_STYLE
    )
    return card

#### Create app layout

In [ ]:
app = dash.Dash(
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

# app = dash.Dash() if you are not in Naas
app.title = APP_TITLE
app.layout = html.Div(
    [
        # Navbar
        navbar,
        
        # HKPIS
        dbc.Row(
            [
                create_card_col("CHIFFRE D'AFFAIRES", "180", "value1", "15", "var1"),
                create_card_col("MARGE BRUTE", "180", "value2", "15", "var2"),
                create_card_col("EBE", "180", "value3", "15", "var3"),
                create_card_col("BFR", "180", "value4", "15", "var4"),
                create_card_col("CREANCES CLIENTS", "180", "value5", "15", "var5"),
                create_card_col("DETTES FOURNISSEURS", "180", "value6", "15", "var6"),
            ],
            style={
                "padding-top": "20px",
            },
            className="g-2 d-flex align-items-center",
        ),
        
        # SPACE
        html.Br(),
        
        # Chart: Row1
        dbc.Row(
            [
                create_chart(
                    "EVOLUTION DU CHIFFRE D'AFFAIRES",
                    "N vs N-1, mensuel (en k€)",
                    "fig_1",
                    fig_linechart,
                    CARD_COL_STYLE3
                ),
                create_chart(
                    "POSITION DE TRESORERIE",
                    "Cash in / Cash out, mensuel (en k€)",
                    "fig_2",
                    fig_barline_chart,
                    CARD_COL_STYLE3,
                ),
            ],
            className="g-2 d-flex align-items-center",
        ),
        
        # SPACE
        html.Br(),
        
        # Chart: Row2
        dbc.Row(
            [
                create_chart(
                    "CHARGES",
                    "N vs N-1 (en k€)",
                    "fig_3",
                    fig_hbarhcart_chart,
                    CARD_COL_STYLE3
                ),
                create_chart(
                    "BILAN",
                    "Structure bilantielle (en k€)",
                    "fig_4",
                    fig_treemap_chart,
                    CARD_COL_STYLE3,
                ),
            ],
            className="g-2 d-flex align-items-center",
        ),
        
        # SPACE
        html.Br(),
    ],
    style={"backgroundColor": BACKGROUND_COLOR}
)

# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# add callback to filter data in cards and charts
@app.callback(
    [
        Output("value1", "children"),
        Output("value2", "children"),
        Output("value3", "children"),
        Output("value4", "children"),
        Output("value5", "children"),
        Output("value6", "children"),
        Output("var1", "children"),
        Output("var2", "children"),
        Output("var3", "children"),
        Output("var4", "children"),
        Output("var5", "children"),
        Output("var6", "children"),
        Output("var1", "style"),
        Output("var2", "style"),
        Output("var3", "style"),
        Output("var4", "style"),
        Output("var5", "style"),
        Output("var6", "style"),
    ],
    [
        Input("entity", "value"),
        Input("scenario", "value")
    ],
)

def multi_outputs(entity, scenario):
    if entity is None and scenario is None:
        raise PreventUpdate

    # Get HKPIs dataframe
    dfhkpis = df_kpis.copy()
    dfhkpis = dfhkpis[
        (dfhkpis["ENTITY"] == entity) & (dfhkpis["PERIOD"].astype(str) == scenario)
    ].reset_index(drop=True)

    # Return card data
    value1 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == "CHIFFRE_D'AFFAIRES", "VALUE_D"].values[0]
    value2 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'MARGE', "VALUE_D"].values[0]
    value3 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'EBE', "VALUE_D"].values[0]
    value4 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'BFR', "VALUE_D"].values[0]
    value5 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'CREANCES_CLIENTS', "VALUE_D"].values[0]
    value6 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'DETTES_FOURNISSEURS', "VALUE_D"].values[0]
    var1 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == "CHIFFRE_D'AFFAIRES", "VARP_D"].values[0]
    var2 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'MARGE', "VARP_D"].values[0]
    var3 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'EBE', "VARP_D"].values[0]
    var4 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'BFR', "VARP_D"].values[0]
    var5 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'CREANCES_CLIENTS', "VARP_D"].values[0]
    var6 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'DETTES_FOURNISSEURS', "VARP_D"].values[0]
    style1 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == "CHIFFRE_D'AFFAIRES", "VARV"].values[0]
    style2 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'MARGE', "VARV"].values[0]
    style3 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'EBE', "VARV"].values[0]
    style4 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'BFR', "VARV"].values[0]
    style5 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'CREANCES_CLIENTS', "VARV"].values[0]
    style6 = dfhkpis.loc[dfhkpis["RUBRIQUE_N1"] == 'DETTES_FOURNISSEURS', "VARV"].values[0]
    
    # Create style on var
    def create_style(value):
        if int(value) < 0:
            color = "#E74C3C"
        elif int(value) == 0:
            color = "#F4D03F"
        elif int(value) > 0:
            color = "#2ECC71"
        return {
            "color": color,
            "font-size": "20px",
            "font-weight": "700",
            "text-align": "center",
            "padding-top": "0px",
        }
    return (
        value1,
        value2,
        value3,
        value4,
        value5,
        value6,
        var1,
        var2,
        var3,
        var4,
        var5,
        var6,
        create_style(style1),
        create_style(style2),
        create_style(style3),
        create_style(style4),
        create_style(style5),
        create_style(style6),
    )


# # color red/green
# @app.callback(
#     [
#         Output("card1", "style"),
#         Output("card2", "style"),
#         Output("card3", "style"),
#         Output("card4", "style"),
#         Output("card5", "style"),
#         Output("card6", "style"),
#     ],
#     [
#         Input("card1", "children"),
#         Input("card2", "children"),
#         Input("card3", "children"),
#         Input("card4", "children"),
#         Input("card5", "children"),
#         Input("card6", "children"),
#     ],
# #     Output("card1", "style"),
# #     Input("card1", "children")
# )
# def update_style(
#     card1,
#     card2,
#     card3,
#     card4,
#     card5,
#     card6
# ):
#     def create_style(value):
#         value = value.split("%")[0].strip()
#         if int(value) < 0:
#             color = "#E74C3C"
#         elif int(value) == 0:
#             color = "#F4D03F"
#         elif int(value) > 0:
#             color = "#2ECC71"
#         return {
#             "color": color,
#             "font-size": "20px",
#             "font-weight": "700",
#             "text-align": "center",
#             "padding-top": "0px",
#         }
#     return create_style(card1), create_style(card2), create_style(card3), create_style(card4), create_style(card5), create_style(card6)

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")